# SALES TERRITORY DIMENSION

In [1]:
import pandas as pd
from connection_script import connect_databases

In [2]:
db_op, db_etl = connect_databases()

## EXTRACT

In [3]:
query = """
        SELECT t1.*, t2.Name AS SalesTerritoryCountry 
        FROM [Sales].[SalesTerritory] AS t1
        INNER JOIN [Person].[CountryRegion] AS t2
        ON t1.CountryRegionCode = t2.CountryRegionCode;
        """

dim_sales_territory = pd.read_sql_query(query, db_op)

## TRANSFORM

In [4]:
dim_sales_territory = dim_sales_territory.drop(columns=['rowguid', 'ModifiedDate', 'SalesYTD', 'SalesLastYear', 'CostYTD', 'CostLastYear', 'CountryRegionCode'])

In [5]:
dim_sales_territory

,TerritoryID,Name,Group,SalesTerritoryCountry
0,1,Northwest,North America,United States
1,2,Northeast,North America,United States
2,3,Central,North America,United States
3,4,Southwest,North America,United States
4,5,Southeast,North America,United States
5,6,Canada,North America,Canada
6,7,France,Europe,France
7,8,Germany,Europe,Germany
8,9,Australia,Pacific,Australia
9,10,United Kingdom,Europe,United Kingdom


In [6]:
# Rename columns
dim_sales_territory = dim_sales_territory.rename(columns={'TerritoryID':'SalesTerritoryAlternateKey',
                                    'Name':'SalesTerritoryRegion', 
                                   'Group': 'SalesTerritoryGroup'})

In [7]:
# Insert empty instance
empty_instance = {
                  'SalesTerritoryAlternateKey':0,
                  'SalesTerritoryRegion':'NA',
                  'SalesTerritoryCountry':'NA',
                  'SalesTerritoryGroup':'NA'
                }

dim_sales_territory = dim_sales_territory._append(empty_instance, ignore_index=True)

In [8]:
# Sort columns
dim_sales_territory = dim_sales_territory[['SalesTerritoryAlternateKey', 'SalesTerritoryRegion', 'SalesTerritoryCountry', 'SalesTerritoryGroup']]

In [9]:
dim_sales_territory

,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup
0,1,Northwest,United States,North America
1,2,Northeast,United States,North America
2,3,Central,United States,North America
3,4,Southwest,United States,North America
4,5,Southeast,United States,North America
5,6,Canada,Canada,North America
6,7,France,France,Europe
7,8,Germany,Germany,Europe
8,9,Australia,Australia,Pacific
9,10,United Kingdom,United Kingdom,Europe


## LOAD

In [10]:
# Sales Territory Image is still missing!!
dim_sales_territory.index += 1
dim_sales_territory.to_sql('DimSalesTerritory', db_etl, if_exists='replace', index_label='SalesTerritoryKey')

11